In [28]:
from rich.console import Console
import sys
import builtins

console = Console(force_terminal=False, file=sys.__stdout__)

builtins.print = console.print

In [ ]:
import json
import os
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from dotenv import load_dotenv, find_dotenv
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults

/home/dgamorim/development/AIagents/.venv/lib/python3.11/site-packages/qdrant_client/http/models/models.py:758: DeprecationWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/home/dgamorim/development/AIagents/.venv/lib/python3.11/site-packages/qdrant_client/http/models/models.py:762: DeprecationWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [2]:
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [9]:
import pathlib

path = pathlib.Path().parent.absolute()
path / 'data'

PosixPath('/home/dgamorim/development/AIagents/CrewAI/data')

In [ ]:
csv_imoveis = CSVSearchTool(csv=str(path / 'data' / '3_data.csv'))
csv_imoveis

CSVSearchTool(name="Search a CSV's content", description='Tool Name: Search a CSV\'s content\nTool Arguments: {\'search_query\': {\'description\': "Mandatory search query you want to use to search the CSV\'s content", \'type\': \'str\'}}\nTool Description: A tool that can be used to semantic search a query the /home/dgamorim/development/AIagents/CrewAI/data/imoveis.csv CSV\'s content.', env_vars=[], args_schema=<class 'crewai_tools.tools.csv_search_tool.csv_search_tool.FixedCSVSearchToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7fd4f81e1080>, result_as_answer=False, max_usage_count=None, current_usage_count=0, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x7fd4e8fdaa10>, summarize=False), config=None)

In [ ]:
# Agente Corretor de Imóveis
corretor_imoveis = Agent(
    role="Corretor de Imóveis",  # define o “papel” do agente, ou seja, como ele deve se comportar 
    goal="Obtenha as preferências do cliente e busque imóveis compatíveis no banco de dados",  # objetivo principal que o agente deve alcançar
    backstory="Especialista no mercado imobiliário, encontra as melhores opções baseadas no perfil do cliente",  # contexto ou “história de fundo” que justifica o expertise do agente
    verbose=True,  # quando True, faz o agente imprimir logs detalhados de cada passo de raciocínio.
    max_iter=5,  # número máximo de iterações/loops que o agente pode executar antes de parar
    tools=[csv_imoveis], # lista de objetos ou funções externas que o agente pode invocar. No seu exemplo, [csv_imoveis] é uma ferramenta para busca em CSV de imóveis.
    allow_delegation=False,  # controla se o agente pode delegar partes da tarefa a outros agentes. False desativa delegação.
    memory=True  # ativa (True) ou não (False) o armazenamento de contexto/memória entre chamadas subsequentes do agente.
)

In [ ]:
# Tarefa - Buscar Imóveis
buscar_imoveis = Task(
    description="Pesquise imóveis na região desejada pelo cliente, considerando faixa de preço e tipo de imóvel",  # descreve o que deve ser feito na tarefa
    expected_output="Lista de imóveis disponíveis com detalhes sobre localização, preço e características",  # texto que define qual formato ou conteúdo de resultado o agente deve retornar
    agent=corretor_imoveis  # instância de Agent responsável por executar essa tarefa
)

In [ ]:
# criando uma tool e alocando a um agent analista_mercado linkando na task

from crewai.tools import BaseTool

def obter_precos_imoveis(cidade: str = "geral"):
    precos = {
        "São Paulo": {"tendencia":"aumento", "percentual":5.2},
        "Rio de Janeiro": {"tendencia":"estavel", "percentual": 0.0},
        "Belo Horizonte": {"tendencia":"queda", "percentual":-3.1},
        "geral": {"tendencia":"aumento", "percentual":4.0} 
    }
    return precos.get(cidade, precos["geral"])

class TendenciaPrecosImoveisTool(BaseTool):
    name: str = "Analisador de Preços Imobiliários"
    description: str = "Obtém tendências de preços de imóveis com base na cidade especificada."
    
    def _run(self, cidade: str) -> dict:
        """
        Executa a análise de preços imobiliários e retorna a tendência com base na cidade.
        """
        try:
            return obter_precos_imoveis(cidade)
        except Exception as e:
            return {"erro": f"Erro ao obter tendências de preços {str(e)}"}

In [17]:
analista_mercado = Agent(
    role="Analista de Mercado Imobiliário",
    goal="Analisa tendências de preços e ajuda a prever a valorização ou desvalorização dos imóveis na cidade {cidade}",
    backstory="Experiente no setor, usa dados históricos para prever preços futuros.",
    verbose=True,
    max_iter=5,
    allow_delegation=False,
    memory=True
)

In [18]:
obter_tendencias = Task(
    description="""
    Analise o histórico de preços de imóveis na cidade {cidade} e forneça insights sobre
    valorização ou desvalorização. Considere o tipo de imóvel {tipo_imovel} e a 
    faixa de preço {faixa_preco}
    """,
    expected_output="Resumo da tendência dos preços no mercado imobiliário",
    tools=[TendenciaPrecosImoveisTool()],
    agent=analista_mercado,
    parameters=["cidade"]
)

In [19]:
analista_noticias = Agent(
    role="Analista de Notícias Imobiliários",
    goal="Busca notícias relevantes sobre o mercado imobiliário para avaliar fatores externos.",
    backstory="Especialista em analisar notícias e tendências econômicas que afetam os preços dos imóveis.",
    verbose=True,
    max_iter=5,
    memory=True
)

In [20]:
searchTool = DuckDuckGoSearchResults(backend="news", num_results=5)

In [21]:
searchTool

DuckDuckGoSearchResults(max_results=5, api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='auto', source='text'), backend='news')

In [22]:
buscar_noticias = Task(
    description=f"Pesquise notícias recentes sobre o mercado imobiliário. Data atual: {datetime.now()}",
    expected_output="Resumo das principais notícias e tendências imobiliárias.",
    agent=analista_noticias,
    tool=[searchTool]
)

In [23]:
consultor_financeiro = Agent(
    role="Consultor Financeiro",
    goal="Analisa opções de financiamento imobiliário com base no perfil do cliente.",
    backstory="Especialista em crédito imobiliário, ajuda clientes a escolherem as melhores opções de financiamento",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True
)

In [24]:
calcular_financiamento = Task(
    description="Analise a renda do cliente e sugira opções de financimaneto viáveis.",
    expected_output="Tabela comparativa com diferentes financiamentos, taxa de juros e prazos",
    agent=consultor_financeiro
)

In [25]:
redator = Agent(
    role="Redator de Relatórios Imobiliários",
    goal="Gera um relatório completo e persuasivo com base nas análises de mercado e imóveis encontrados.",
    backstory="Especialista em comunicação, traduz dados complexos para clientes de forma clara e objetiva.",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True
)

In [26]:
gerar_relatorio = Task(
    description="Gere um relatório detalhado sobre o melhor imóvel encontrado, considerando preços, tendências e financiamento.",
    expected_output="Relatório formatado com resumo do mercado, opções recomendadas e justificativa da escolha.",
    agent=redator,
    context=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financiamento]
)

In [ ]:
crew = Crew(
    agents=[corretor_imoveis, analista_mercado, analista_noticias, consultor_financeiro, redator],  # lista de instâncias de Agent que compõem a equipe e executarão as tarefas.
    tasks=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financiamento, gerar_relatorio],  # lista de instâncias de Task que serão atribuídas aos agentes para execução.
    verbose=True,  # faz a Crew logar passo a passo de tudo que acontece.
    process=Process.hierarchical,  #  estratégia de orquestração entre agentes e tarefas. Ex.: Process.hierarchical faz um fluxo hierárquico (um agente gerencia subtarefas pelos demais).
    full_output=True,  # faz a Crew retornar todo o histórico de interações e outputs intermediários, não só o resultado final.
    share_crew=False,  # isola o estado/memória de cada agente entre execuções da Crew; True faz agentes compartilharem contexto.
    max_iter=15,  # com o número máximo de ciclos (iterações) que a Crew pode rodar antes de interromper o processo.
    manager_llm=llm  # instância de LLM usada pelo “gerente” da Crew para tomar decisões de roteamento e síntese entre agentes.
)

In [29]:
result = crew.kickoff(inputs={"cidade": "Rio de Janeiro",
                              "tipo_imovel": "Apartamento",
                              "faixa_preco": "500000-700000"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ec5c86b8-84fb-42fb-aa68-3224e753e489                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[1m[95m# Agent:[00m [1m[92mCrew Manager[00m

[95m## Task:[00m [92mPesquise imóveis na região desejada pelo cliente, considerando faixa de preço e tipo de 
imóvel[00m

Output()

/home/dgamorim/development/AIagents/.venv/lib/python3.11/site-packages/chromadb/types.py:144: 
PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you 
should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

...

[95m## Tool Output:[00m [92m
**Relatório de Análise de Propriedades – Setembro de 2023**

**Propriedade em Destaque: Cobertura em Curitiba**  
- **Endereço:** Rua XV de Novembro, 789 - Curitiba  
- **Preço:** R$ 1.318.826  
- **Quartos:** 4  
- **Banheiros:** 1  
- **Metragem:** 75 m²  
- **Tipo:** Cobertura  

**Análise de Mercado:**  
Após uma análise detalhada do mercado imobiliário atual, a cobertura localizada na Rua XV de Novembro, 789, em 
Curitiba, se destaca como uma excelente oportunidade de investimento. Atualmente, a demanda por propriedades na 
região Centro de Curitiba está crescente, e essa cobertura oferece um bom equilíbrio entre preço e metragem.

Com o preço de R$ 1.318.826, seu custo por metro quadrado é de aproximadamente R$ 17.570, que é competitivo 
considerando o espaço e a localização. Adicionalmente, a cobertura possui 4 quartos, o que a torna ideal para 
famílias que buscam espaço e conforto. O único banheiro pode ser uma desvantagem, mas a possibilidade de 
remodelação é uma alternativa atrativa para os compradores.

**Comparação com Outras Propriedades:**  
Ao analisar outras coberturas disponíveis na mesma localidade, encontramos outra propriedade na Rua XV de Novembro,
próxima a esta, com 5 quartos e 3 banheiros, porém, o preço é significativamente mais elevado, totalizando R$ 
1.789.771. Embora tenha mais comodidades, a diferença de preço pode afastar potenciais compradores que buscam um 
investimento mais acessível.

Comparando com uma propriedade em São Paulo, na Avenida Paulista, 101, que tem 3 quartos e 4 banheiros, e custa R$ 
804.555, percebemos que, apesar de São Paulo ter um mercado aquecido, o tamanho da propriedade (118 m²) proporciona
um valor por metro quadrado mais baixo, mas, considerando o perfil dos compradores, a cobertura em Curitiba se 
mostra mais estratégica.

**Tendências de Preço:**  
Os preços das propriedades em Curitiba atualmente demonstram uma alta em relação ao ano anterior, com previsão de 
continuidade dessa tendência, especialmente dada a urbanização da área e o investimento em infraestrutura. 
Portanto, a aquisição da cobertura pode ser vantajosa tanto para moradia quanto para investimento a longo prazo.

**Opções de Financiamento:**  
Aproximadamente 80% das compras de imóveis são realizadas através de financiamento. Portanto, é essencial 
considerar as opções disponíveis para os potenciais compradores. A cobertura pode ser financiada através de 
instituições que oferecem taxas de 7% a 10% ao ano, com planos de pagamento que variam de 15 a 30 anos. 
Recomendamos que os compradores façam simulações para escolha do melhor plano que se encaixe em suas necessidades e
capacidade financeira.

**Conclusão:**  
Diante de todos os fatores analisados, a cobertura na Rua XV de Novembro, 789, em Curitiba, se posiciona como uma 
das melhores opções atualmente disponíveis no mercado. Seu preço competitivo, somado ao potencial de valorização e 
opções de financiamento, torna-a bastante atrativa. Recomendo que avancemos com a promoção dessa propriedade, 
destacando suas características e os benefícios de se adquirir um imóvel nesse local.[00m

[1m[95m# Agent:[00m [1m[92mCrew Manager[00m

[95m## Final Answer:[00m [92m
**Relatório de Análise de Propriedades – Setembro de 2023**

**Propriedade em Destaque: Cobertura em Curitiba**  
- **Endereço:** Rua XV de Novembro, 789 - Curitiba  
- **Preço:** R$ 1.318.826  
- **Quartos:** 4  
- **Banheiros:** 1  
- **Metragem:** 75 m²  
- **Tipo:** Cobertura  

**Análise de Mercado:**  
Após uma análise detalhada do mercado imobiliário atual, a cobertura localizada na Rua XV de Novembro, 789, em 
Curitiba, se destaca como uma excelente oportunidade de investimento. Atualmente, a demanda por propriedades na 
região Centro de Curitiba está crescente, e essa cobertura oferece um bom equilíbrio entre preço e metragem.

Com o preço de R$ 1.318.826, seu custo por metro quadrado é de aproximadamente R$ 17.570, que é competitivo 
considerando o espaço e a localização. Adicionalmente, a cobertura possui 4 quartos, o que a torna ideal para 
famílias que buscam espaço e conforto. O único banheiro pode ser uma desvantagem, mas a possibilidade de 
remodelação é uma alternativa atrativa para os compradores.

**Comparação com Outras Propriedades:**  
Ao analisar outras coberturas disponíveis na mesma localidade, encontramos outra propriedade na Rua XV de Novembro,
próxima a esta, com 5 quartos e 3 banheiros, porém, o preço é significativamente mais elevado, totalizando R$ 
1.789.771. Embora tenha mais comodidades, a diferença de preço pode afastar potenciais compradores que buscam um 
investimento mais acessível.

Comparando com uma propriedade em São Paulo, na Avenida Paulista, 101, que tem 3 quartos e 4 banheiros, e custa R$ 
804.555, percebemos que, apesar de São Paulo ter um mercado aquecido, o tamanho da propriedade (118 m²) proporciona
um valor por metro quadrado mais baixo, mas, considerando o perfil dos compradores, a cobertura em Curitiba se 
mostra mais estratégica.

**Tendências de Preço:**  
Os preços das propriedades em Curitiba atualmente demonstram uma alta em relação ao ano anterior, com previsão de 
continuidade dessa tendência, especialmente dada a urbanização da área e o investimento em infraestrutura. 
Portanto, a aquisição da cobertura pode ser vantajosa tanto para moradia quanto para investimento a longo prazo.

**Opções de Financiamento:**  
Aproximadamente 80% das compras de imóveis são realizadas através de financiamento. Portanto, é essencial 
considerar as opções disponíveis para os potenciais compradores. A cobertura pode ser financiada através de 
instituições que oferecem taxas de 7% a 10% ao ano, com planos de pagamento que variam de 15 a 30 anos. 
Recomendamos que os compradores façam simulações para escolha do melhor plano que se encaixe em suas necessidades e
capacidade financeira.

**Conclusão:**  
Diante de todos os fatores analisados, a cobertura na Rua XV de Novembro, 789, em Curitiba, se posiciona como uma 
das melhores opções atualmente disponíveis no mercado. Seu preço competitivo, somado ao potencial de valorização e 
opções de financiamento, torna-a bastante atrativa. Recomendo que avancemos com a promoção dessa propriedade, 
destacando suas características e os benefícios de se adquirir um imóvel nesse local.[00m

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bac8a886-987e-4e6d-bde3-cf1a5419f5d5                                                                     │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: ec5c86b8-84fb-42fb-aa68-3224e753e489                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [30]:
from IPython.display import display, Markdown

In [31]:
display(Markdown(str(result)))

**Relatório de Análise de Propriedades – Setembro de 2023**

**Propriedade em Destaque: Cobertura em Curitiba**  
- **Endereço:** Rua XV de Novembro, 789 - Curitiba  
- **Preço:** R$ 1.318.826  
- **Quartos:** 4  
- **Banheiros:** 1  
- **Metragem:** 75 m²  
- **Tipo:** Cobertura  

**Análise de Mercado:**  
Após uma análise detalhada do mercado imobiliário atual, a cobertura localizada na Rua XV de Novembro, 789, em Curitiba, se destaca como uma excelente oportunidade de investimento. Atualmente, a demanda por propriedades na região Centro de Curitiba está crescente, e essa cobertura oferece um bom equilíbrio entre preço e metragem.

Com o preço de R$ 1.318.826, seu custo por metro quadrado é de aproximadamente R$ 17.570, que é competitivo considerando o espaço e a localização. Adicionalmente, a cobertura possui 4 quartos, o que a torna ideal para famílias que buscam espaço e conforto. O único banheiro pode ser uma desvantagem, mas a possibilidade de remodelação é uma alternativa atrativa para os compradores.

**Comparação com Outras Propriedades:**  
Ao analisar outras coberturas disponíveis na mesma localidade, encontramos outra propriedade na Rua XV de Novembro, próxima a esta, com 5 quartos e 3 banheiros, porém, o preço é significativamente mais elevado, totalizando R$ 1.789.771. Embora tenha mais comodidades, a diferença de preço pode afastar potenciais compradores que buscam um investimento mais acessível.

Comparando com uma propriedade em São Paulo, na Avenida Paulista, 101, que tem 3 quartos e 4 banheiros, e custa R$ 804.555, percebemos que, apesar de São Paulo ter um mercado aquecido, o tamanho da propriedade (118 m²) proporciona um valor por metro quadrado mais baixo, mas, considerando o perfil dos compradores, a cobertura em Curitiba se mostra mais estratégica.

**Tendências de Preço:**  
Os preços das propriedades em Curitiba atualmente demonstram uma alta em relação ao ano anterior, com previsão de continuidade dessa tendência, especialmente dada a urbanização da área e o investimento em infraestrutura. Portanto, a aquisição da cobertura pode ser vantajosa tanto para moradia quanto para investimento a longo prazo.

**Opções de Financiamento:**  
Aproximadamente 80% das compras de imóveis são realizadas através de financiamento. Portanto, é essencial considerar as opções disponíveis para os potenciais compradores. A cobertura pode ser financiada através de instituições que oferecem taxas de 7% a 10% ao ano, com planos de pagamento que variam de 15 a 30 anos. Recomendamos que os compradores façam simulações para escolha do melhor plano que se encaixe em suas necessidades e capacidade financeira.

**Conclusão:**  
Diante de todos os fatores analisados, a cobertura na Rua XV de Novembro, 789, em Curitiba, se posiciona como uma das melhores opções atualmente disponíveis no mercado. Seu preço competitivo, somado ao potencial de valorização e opções de financiamento, torna-a bastante atrativa. Recomendo que avancemos com a promoção dessa propriedade, destacando suas características e os benefícios de se adquirir um imóvel nesse local.